In [31]:

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
import numpy as np
from mlflow.models.signature import infer_signature
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [41]:
load_dotenv()

access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_url = os.getenv("MLFLOW_S3_ENDPOINT_URL")
tracker_url = os.getenv("MLFLOW_URL")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_ADMIN_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_ADMIN_PASSWORD")
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
bucket_name = "dataset"
object_name = "water_potability.csv"

In [33]:
s3 = boto3.client(
    's3',
    endpoint_url=s3_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    verify=False
)

In [34]:
try:
    # Attempt to list buckets
    response = s3.list_buckets()
    
    # If successful, print bucket names
    print("Connected successfully! Buckets available:")
    for bucket in response['Buckets']:
        print(f"- {bucket['Name']}")
except Exception as e:
    # Print any connection errors
    print("Connection error:", e)


Connected successfully! Buckets available:
- dataset
- mlflow


In [35]:
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    print(response)
    dataset_content = response.get('Body')
    print(dataset_content)
    # Load the dataset into a DataFrame
    df = pd.read_csv(dataset_content)
    print("Dataset loaded successfully:")
    print(df.head())
except Exception as e:
    print("Error fetching dataset from MinIO:", e)


{'ResponseMetadata': {'RequestId': '180762EFD6F40820', 'HostId': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-length': '525187', 'content-type': 'text/csv', 'etag': '"a5eeae83d792e40da6c2f9e1eab945fc"', 'last-modified': 'Tue, 12 Nov 2024 07:15:22 GMT', 'server': 'MinIO', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'x-amz-request-id': '180762EFD6F40820', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '851', 'x-ratelimit-remaining': '851', 'x-xss-protection': '1; mode=block', 'x-amz-version-id': 'ae37a965-df07-433c-85ff-3d7ac1b462ef', 'date': 'Wed, 13 Nov 2024 01:29:23 GMT'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2024, 11, 12, 7, 15, 22, tzinfo=tzutc()), 'ContentLength': 525187, 'ETag': '"a5eeae83d792e

In [36]:
df = df[['ph','Solids','Potability']]

In [37]:
df

,ph,Solids,Potability
0,NaN,20791.318981,0
1,3.716080,18630.057858,0
2,8.099124,19909.541732,0
3,8.316766,22018.417441,0
4,9.092223,17978.986339,0
...,...,...,...
3271,4.668102,47580.991603,1
3272,7.808856,17329.802160,1
3273,9.419510,33155.578218,1
3274,5.126763,11983.869376,1


In [38]:
mlflow.set_tracking_uri(tracker_url)
experiment_name = "water_potability"
mlflow.set_experiment(experiment_name)

data = df.drop('Potability', axis=1)
target = df['Potability']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [42]:
with mlflow.start_run():
    n_estimators = 100
    max_depth = 5
    random_state = 42

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    mlflow.log_metric("accuracy", accuracy)

    input_example = X_test.iloc[1].fillna(X_test.mean()).values.reshape(1, -1)
    input_example = np.array([X_test.iloc[0].fillna(X_test.mean())])
    signature = infer_signature(X_test, predictions)

    mlflow.xgboost.log_model(model, "model", input_example=input_example, signature=signature)

    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2024/11/13 08:31:18 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/13 08:31:18 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:31:19] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1724807611129/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/11/13 08:31:23 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      7.04828946039254,
      20461.252710219946
    ]
  ]
}. Alternatively, you can avoid passing input example and pass model signature inste

Run ID: 7ef2b4c3d4dc459ab28b1aa0bab68b36
Model accuracy: 0.573170731707317


2024/11/13 08:31:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run resilient-robin-194 at: http://10.34.4.198:5000/#/experiments/1/runs/7ef2b4c3d4dc459ab28b1aa0bab68b36.
2024/11/13 08:31:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://10.34.4.198:5000/#/experiments/1.
2024/11/13 08:31:24 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/13 08:31:24 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [43]:
with mlflow.start_run():
    n_estimators = 100
    max_depth = 5
    random_state = 42

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    mlflow.log_metric("accuracy", accuracy)

    input_example = X_test.iloc[1].fillna(X_test.mean()).values.reshape(1, -1)
    input_example = np.array([X_test.iloc[0].fillna(X_test.mean())])
    signature = infer_signature(X_test, predictions)

    mlflow.sklearn.log_model(model, "model", input_example=input_example, signature=signature)

    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2024/11/13 08:31:24 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/13 08:31:24 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/11/13 08:31:29 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      7.04828946039254,
      20461.252710219946
    ]
  ]
}. Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: Failed to enforce schema of data '[[7.04828946e+00 2.04612527e+04

Run ID: b3c173ba76a14d9caec103c1b1068203
Model accuracy: 0.6234756097560976


2024/11/13 08:31:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run bittersweet-lamb-805 at: http://10.34.4.198:5000/#/experiments/1/runs/b3c173ba76a14d9caec103c1b1068203.
2024/11/13 08:31:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://10.34.4.198:5000/#/experiments/1.
2024/11/13 08:31:30 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/13 08:31:30 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
